In [1]:
import pandas as pd
import numpy as np

In [2]:
S = pd.read_csv('dataset_prepped')

In [3]:
max_id = 1178409
S = S[S.innings == 1][S.match_id < max_id]

C:\Users\alfie\AppData\Local\Temp\ipykernel_6080\4171573495.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  S = S[S.innings == 1][S.match_id < max_id]


In [4]:
H = pd.read_csv('../IPL_Dataset/1178409_info.csv', nrows=20, index_col=1)

In [5]:
H

,version,2.1.0
balls_per_over,info,6
team,info,Mumbai Indians
team,info,Delhi Capitals
gender,info,male
season,info,2019
date,info,2019/04/18
event,info,Indian Premier League
match_number,info,34
venue,info,Arun Jaitley Stadium
city,info,Delhi


In [6]:
H.loc['city'][1]

'Delhi'

In [7]:
for i in list(S.index):
    if pd.isnull(S.loc[i,'wides']):
        S.loc[i,'wides'] = 0
    if pd.isnull(S.loc[i,'noballs']):
        S.loc[i,'noballs'] = 0

In [8]:
S

,match_id,innings,over,wickets,striker,bowler,delivery,wides,noballs,batting_outcome,fielding_extras,fielding_wicket,free_hit
0,335982,1,1,0.0,SC Ganguly,P Kumar,fair,0.0,0.0,0,1.0,False,False
1,335982,1,1,0.0,BB McCullum,P Kumar,fair,0.0,0.0,0,0.0,False,False
2,335982,1,1,0.0,BB McCullum,P Kumar,wide,1.0,0.0,0,0.0,False,False
3,335982,1,1,0.0,BB McCullum,P Kumar,fair,0.0,0.0,0,0.0,False,False
4,335982,1,1,0.0,BB McCullum,P Kumar,fair,0.0,0.0,0,0.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
172624,1136620,1,20,5.0,YK Pathan,SN Thakur,fair,0.0,0.0,0,0.0,False,False
172625,1136620,1,20,5.0,YK Pathan,SN Thakur,fair,0.0,0.0,1,0.0,False,False
172626,1136620,1,20,5.0,CR Brathwaite,SN Thakur,fair,0.0,0.0,0,0.0,False,False
172627,1136620,1,20,5.0,CR Brathwaite,SN Thakur,fair,0.0,0.0,6,0.0,False,False


In [9]:
total_runs = []
team1 = []
opp = []
city = []
tossw = []
matches = sorted(list(set(S.match_id)))
for idx in matches:
    inns = pd.read_csv('../IPL_Dataset/'+ str(idx) + '.csv')
    inns = inns[inns.innings == 1]
    info = pd.read_csv('../IPL_Dataset/'+ str(idx) + '_info.csv', nrows=18, index_col=1)

    total_runs.append(np.nansum(inns.runs_off_bat) + np.nansum(inns.extras))

    team1.append(inns.loc[0, 'batting_team'])
    opp.append(inns.loc[0, 'bowling_team'])

    city.append(info.loc['city'][1])
    tossw.append(info.loc['toss_winner'][1] == inns.loc[0, 'batting_team'])



In [10]:
df = pd.DataFrame(matches, columns=['MatchID'])
df['Team'] = team1
df['Opponent'] = opp
df['City'] = city
df['Elected'] = tossw
df['Runs'] = total_runs


In [11]:
for i in range(len(df)):
    if df.loc[i,'Team'] == 'Rising Pune Supergiant':
        df.loc[i,'Team'] = 'Rising Pune Supergiants'
    if df.loc[i,'Team'] == 'Delhi Daredevils':
        df.loc[i,'Team'] = 'Delhi Capitals'
    if df.loc[i,'Team'] == 'Deccan Chargers':
        df.loc[i,'Team'] = 'Sunrisers Hyderabad'
    if df.loc[i,'Opponent'] == 'Rising Pune Supergiant':
        df.loc[i,'Opponent'] = 'Rising Pune Supergiants'
    if df.loc[i,'Opponent'] == 'Delhi Daredevils':
        df.loc[i,'Opponent'] = 'Delhi Capitals'
    if df.loc[i,'Opponent'] == 'Deccan Chargers':
        df.loc[i,'Opponent'] = 'Sunrisers Hyderabad'



In [12]:
tc = {}
for c in list(set(df.City)):
    if type(c) == str:
        for t in list(set(df.Team)):
            if c in t:
                tc[t] = c

In [13]:
# GL, KXI, RR: rajkot, chandigarh, jaipur, 

tc['Gujarat Lions'] = 'Rajkot'
tc['Kings XI Punjab'] = 'Chandigarh'
tc['Rajasthan Royals'] = 'Jaipur'

In [14]:
def han(team1, opp, city):
    if tc[team1] == city:
        return 'Home'
    if tc[opp] == city:
        return 'Away'
    else:
        return 'Neutral'

In [15]:
df['H/A'] = list(map(han, df['Team'], df['Opponent'], df['City']))

In [16]:
df

,MatchID,Team,Opponent,City,Elected,Runs,H/A
0,335982,Kolkata Knight Riders,Royal Challengers Bangalore,Bangalore,False,222,Away
1,335983,Chennai Super Kings,Kings XI Punjab,Chandigarh,True,240,Away
2,335984,Rajasthan Royals,Delhi Capitals,Delhi,True,129,Away
3,335985,Mumbai Indians,Royal Challengers Bangalore,Mumbai,True,165,Home
4,335986,Sunrisers Hyderabad,Kolkata Knight Riders,Kolkata,True,110,Away
...,...,...,...,...,...,...,...
724,1178404,Kolkata Knight Riders,Chennai Super Kings,Kolkata,False,161,Home
725,1178405,Delhi Capitals,Sunrisers Hyderabad,Hyderabad,False,155,Away
726,1178406,Royal Challengers Bangalore,Mumbai Indians,Mumbai,False,171,Away
727,1178407,Kings XI Punjab,Rajasthan Royals,Chandigarh,False,182,Home


In [17]:
h = df[df['H/A'] == 'Home'].Runs.mean()

In [18]:
n = df[df['H/A'] == 'Neutral'].Runs.mean()

In [19]:
a = df[df['H/A'] == 'Away'].Runs.mean()

In [20]:
df[df['H/A'] == 'Away'].Runs.std()

32.09853971334066

In [21]:
df[df.Elected].Runs.mean()

157.72473867595818

In [22]:
df[df.Elected == False].Runs.mean()

163.23529411764707

In [23]:
2*h/(a+h)

1.0223137477253796

In [24]:
2*a/(a+h)

0.9776862522746202

In [25]:
len(df[df.Elected])

287